# Gene essentiality

In [2]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from bacillusme.util.helper_functions import *

%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/__init__.py
/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/bacillusme/__init__.py


In [3]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f06a6b01978>

In [3]:
with open('../me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

## Essentiality

M-model

In [42]:
from ecolime.analysis import essentiality as es

In [59]:
genes = [gene.id for gene in bsub.genes]
essentiality_df_m,flux_responses_m = es.gene_essentiality(bsub,genes, 'm',NP=10,threshold = 0.01,precision=1e-6)

10 threads: 100%|██████████| 844/844 [00:59<00:00, 16.10it/s]

Done


In [70]:
flux_responses_m.to_csv('essentiality_results/essentiality_raw_results.csv')

ME-model

In [83]:
genes = [gene.id for gene in me.genes]
essentiality_df_me,flux_responses_me = es.gene_essentiality(me,genes,'me',NP=10,threshold = 0.01,precision=1e-6)

Done


In [85]:
flux_responses_me.to_csv('essentiality_results/essentiality_raw_me_results.csv')

### Save combined results

In [96]:
essentiality_df = essentiality_df_me.join(essentiality_df_m,lsuffix='_me',rsuffix='_m')

In [97]:
essentiality_df.to_csv('essentiality_results/essentiality_results.csv')

### Read

In [25]:
# Read
essentiality_df = pd.read_csv('essentiality_results.csv',index_col=0)

## M/ME comparison

In [105]:
essentiality_df[essentiality_df.essentiality_m != essentiality_df.essentiality_me].dropna()

,response_me,essentiality_me,response_m,essentiality_m
BSU06420,0.000000,0,-1.000000e+00,e
BSU06430,0.000000,0,-1.000000e+00,e
BSU06470,0.000000,0,-1.000000e+00,e
BSU06480,0.000000,0,-1.000000e+00,e
BSU11610,0.000000,0,-1.000000e+00,e
BSU14580,-0.000028,0,-3.526427e-02,-
BSU14590,-0.000028,0,-3.526427e-02,-
BSU14600,-0.000009,0,-3.526427e-02,-
BSU22440,0.000000,0,-1.000000e+00,e
BSU22680,0.000000,0,-1.000000e+00,e
